In [68]:
import torch
import pyro
import pyro.distributions as dist
from pyro.infer import MCMC, NUTS
from sklearn.metrics import accuracy_score, roc_auc_score

# Function to load dataset
def load_data(data_path, labels_path, sep='\s+', header=None, label_transform={-1: 0}):
    """Loads data and labels"""
    data = pd.read_csv(data_path, header=header, sep=sep)
    labels = pd.read_csv(labels_path, header=header, sep=sep).replace(label_transform)
    return data, labels

# Load training and validation datasets
train_data, train_labels = load_data(train_data_path, train_labels_path)
valid_data, valid_labels = load_data(valid_data_path, valid_labels_path)

# Combine training and validation datasets
X = pd.concat([train_data, valid_data], ignore_index=True)
y = pd.concat([train_labels, valid_labels], ignore_index=True)

# Prepare for stratified 5-fold cross-validation
kf = KFold(n_splits=5)

In [67]:
# Initialize lists to store the results
accuracies = []
aucs = []
ras = []
rbs = []

for train_index, test_index in kf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # tensor 
    X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
    X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train.values.flatten(), dtype=torch.float32)
    
    # logistic regression model with gaussian prior
    def model(X, y):
        beta = pyro.sample("beta", dist.Normal(0., 1.).expand([X.shape[1]]).to_event(1))
        y_prob = torch.sigmoid(torch.matmul(X, beta))
        with pyro.plate("data", X.shape[0]):
            pyro.sample("y", dist.Bernoulli(y_prob), obs=y)
    
    # NUTS sampler
    nuts_kernel = NUTS(model)
    mcmc = MCMC(nuts_kernel, num_samples=2000, warmup_steps=1000)
    mcmc.run(X_train_tensor, y_train_tensor)
    
    # sample from the posterior
    beta_sample = mcmc.get_samples()["beta"].detach().numpy()
    y_sample = torch.sigmoid(torch.matmul(X_test_tensor, torch.tensor(beta_sample.T, dtype=torch.float32))).detach().numpy()
    
    # compute the posterior mean and the 95% credible interval
    y_prob = np.mean(y_sample, axis=1)
    y_pred = np.round(y_prob)
    posterior_lower = np.percentile(y_sample, 2.5, axis=1)
    posterior_upper = np.percentile(y_sample, 97.5, axis=1)
    
    acc = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob)
    RA = np.sum((( (np.ravel(y_test.values)  != y_pred) & (posterior_upper > 0.5) & (posterior_lower < 0.5)))) / np.sum(y_test.values != y_pred)
    RB = np.sum((( (np.ravel(y_test.values) == y_pred) & ((posterior_upper <= 0.5) | (posterior_lower >= 0.5))))) / np.sum((posterior_upper <= 0.5) | (posterior_lower >= 0.5))
    accuracies.append(acc)
    aucs.append(auc)
    ras.append(RA)
    rbs.append(RB)

# output
print("Average Accuracy:", np.mean(accuracies))
print("Average AUC:", np.mean(aucs))
print("Average RA:", np.mean(ras))
print("Average RB:", np.mean(rbs))

Sample: 100%|██████████| 3000/3000 [2:17:08,  2.74s/it, step size=2.15e-04, acc. prob=0.953] 

Average Accuracy: 0.805
Average AUC: 0.8863581168831169
Average RA: 0.008635009613354009
Average RB: 0.9361344537815125


In [69]:
# Initialize lists to store the results
accuracies = []
aucs = []
ras = []
rbs = []

for train_index, test_index in kf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # tensor 
    X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
    X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train.values.flatten(), dtype=torch.float32)
    
    # logistic regression model with laplace prior
    def model(X, y):
        beta = pyro.sample("beta", dist.Laplace(0., 1.).expand([X.shape[1]]).to_event(1))
        y_prob = torch.sigmoid(torch.matmul(X, beta))
        with pyro.plate("data", X.shape[0]):
            pyro.sample("y", dist.Bernoulli(y_prob), obs=y)
    
    # NUTS sampler
    nuts_kernel = NUTS(model)
    mcmc = MCMC(nuts_kernel, num_samples=2000, warmup_steps=1000)
    mcmc.run(X_train_tensor, y_train_tensor)
    
    # sample from the posterior
    beta_sample = mcmc.get_samples()["beta"].detach().numpy()
    y_sample = torch.sigmoid(torch.matmul(X_test_tensor, torch.tensor(beta_sample.T, dtype=torch.float32))).detach().numpy()
    
    # compute the posterior mean and the 95% credible interval
    y_prob = np.mean(y_sample, axis=1)
    y_pred = np.round(y_prob)
    posterior_lower = np.percentile(y_sample, 2.5, axis=1)
    posterior_upper = np.percentile(y_sample, 97.5, axis=1)
    
    acc = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob)
    RA = np.sum((( (np.ravel(y_test.values)  != y_pred) & (posterior_upper > 0.5) & (posterior_lower < 0.5)))) / np.sum(y_test.values != y_pred)
    RB = np.sum((( (np.ravel(y_test.values) == y_pred) & ((posterior_upper <= 0.5) | (posterior_lower >= 0.5))))) / np.sum((posterior_upper <= 0.5) | (posterior_lower >= 0.5))
    accuracies.append(acc)
    aucs.append(auc)
    ras.append(RA)
    rbs.append(RB)

# output
print("Average Accuracy:", np.mean(accuracies))
print("Average AUC:", np.mean(aucs))
print("Average RA:", np.mean(ras))
print("Average RB:", np.mean(rbs))

Sample: 100%|██████████| 3000/3000 [43:56,  1.14it/s, step size=2.49e-04, acc. prob=0.935]

Average Accuracy: 0.8150000000000001
Average AUC: 0.8839054653679653
Average RA: 0.008193325125770798
Average RB: 0.9361111111111111
